In [4]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import os
import langid
import time
import ast
import timeit

#!pip install transformers torch --break-system-packages
from transformers import pipeline

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)

In [5]:
movies_sample_path = os.path.abspath("../.data/movies_cleaned.csv")
raitings_path = os.path.abspath("../.data/ratings_cleaned.csv")
credits_path = os.path.abspath("../.data/credits_cleaned.csv")
movies_df = pd.read_csv(movies_sample_path, low_memory=False)
ratings_df = pd.read_csv(raitings_path)
credits_df = pd.read_csv(credits_path)

In [6]:
#transforming to datetime type

movies_df['release_date'] = pd.to_datetime(movies_df['release_date'])
ratings_df['timestamp'] = pd.to_datetime(ratings_df['timestamp'])

In [7]:
movies_df["id"].nunique()

44405

In [8]:
movies_df.head()

,id,title,genres,release_date,overview,production_countries,spoken_languages,budget,revenue
0,862,Toy Story,"['Animation', 'Comedy', 'Family']",1995-10-30,"Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear o...",['United States of America'],['English'],30000000,373554033.0
1,8844,Jumanji,"['Adventure', 'Fantasy', 'Family']",1995-12-15,When siblings Judy and Peter discover an enchanted board game that opens the door to a magical w...,['United States of America'],"['English', 'Français']",65000000,262797249.0
2,15602,Grumpier Old Men,"['Romance', 'Comedy']",1995-12-22,A family wedding reignites the ancient feud between next-door neighbors and fishing buddies John...,['United States of America'],['English'],0,0.0
3,31357,Waiting to Exhale,"['Comedy', 'Drama', 'Romance']",1995-12-22,"Cheated on, mistreated and stepped on, the women are holding their breath, waiting for the elusi...",['United States of America'],['English'],16000000,81452156.0
4,11862,Father of the Bride Part II,['Comedy'],1995-02-10,"Just when George Banks has recovered from his daughter's wedding, he receives the news that she'...",['United States of America'],['English'],0,76578911.0


In [9]:
movies_df.dtypes

id                               int64
title                           object
genres                          object
release_date            datetime64[ns]
overview                        object
production_countries            object
spoken_languages                object
budget                           int64
revenue                        float64
dtype: object

In [10]:
ratings_df.dtypes

movieId               int64
rating              float64
timestamp    datetime64[ns]
dtype: object

In [11]:
ratings_df.head()

,movieId,rating,timestamp
0,110,1.0,2015-03-09 22:52:09
1,147,4.5,2015-03-09 23:07:15
2,858,5.0,2015-03-09 22:52:03
3,1221,5.0,2015-03-09 22:52:26
4,1246,5.0,2015-03-09 22:52:36


In [12]:
credits_df.dtypes

id       int64
cast    object
dtype: object

In [13]:
credits_df.head()

,id,cast
0,862,"['Tom Hanks', 'Tim Allen', 'Don Rickles', 'Jim Varney', 'Wallace Shawn', 'John Ratzenberger', 'A..."
1,8844,"['Robin Williams', 'Jonathan Hyde', 'Kirsten Dunst', 'Bradley Pierce', 'Bonnie Hunt', 'Bebe Neuw..."
2,15602,"['Walter Matthau', 'Jack Lemmon', 'Ann-Margret', 'Sophia Loren', 'Daryl Hannah', 'Burgess Meredi..."
3,31357,"['Whitney Houston', 'Angela Bassett', 'Loretta Devine', 'Lela Rochon', 'Gregory Hines', 'Dennis ..."
4,11862,"['Steve Martin', 'Diane Keaton', 'Martin Short', 'Kimberly Williams-Paisley', 'George Newbern', ..."


## Resolucion de Queries

In [14]:
# Primero buscamos filtrar por anio, ya que es un filtro que utilizan mas de una Query

def filter_by_year(year_min, year_max, df):
    df_filtered = df[df["release_date"].dt.year >= year_min]
    if year_max > year_min:
        df_filtered = df_filtered[df_filtered["release_date"].dt.year < year_max]

    return df_filtered


In [15]:
# Filtar por Pais
def filter_by_country(countries, df):
    df_filtered = df[df["production_countries"].apply(
        lambda x: all(country.lower() in str(x).lower() for country in countries)
    )]
    
    return df_filtered

#### Resolucion Query1: Peliculas y sus géneros de los años 00' con producción Argentina y Española

In [16]:
# aplicando los filtros
movies_2000s_df = filter_by_year(2000, 2010, movies_df)
countries = ["Argentina", "Spain"]
movies_and_genres_2000_Arg_Esp = filter_by_country(countries, movies_2000s_df)
q1_df = movies_and_genres_2000_Arg_Esp.dropna(subset=["title", "genres"])[["title", "genres"]].copy()
q1_df.shape

(24, 2)

In [17]:

q1_df

,title,genres
4672,La Cienaga,"['Comedy', 'Drama']"
4716,Burnt Money,['Crime']
7941,The City of No Limits,"['Thriller', 'Drama']"
9353,Nicotina,"['Drama', 'Action', 'Comedy', 'Thriller']"
9404,Lost Embrace,"['Drama', 'Foreign']"
9919,Whisky,"['Comedy', 'Drama', 'Foreign']"
10290,The Holy Girl,"['Drama', 'Foreign']"
10602,The Aura,"['Crime', 'Drama', 'Thriller']"
10702,Bombón: The Dog,['Drama']
11467,Rolling Family,"['Drama', 'Comedy']"


In [18]:
#q1_df.to_csv('output_q1.csv', index=True)

#### Resolucion Query 2: Top 5 de países que más dinero han invertido en producciones sin colaborar con otros paises

In [19]:
def solo_country_investements(df):
    solo_country_df = df[df['production_countries'].apply(lambda x: len(eval(x)) == 1)].copy()
    solo_country_df.loc[:, 'country'] = solo_country_df['production_countries'].apply(lambda x: eval(x)[0])
    investment_by_country = solo_country_df.groupby('country')['budget'].sum().sort_values(ascending=False)

    return investment_by_country

In [20]:
# Busco la cantidad de inversiones que hizo cada pais, invirtiendo solo
invesments_by_country = solo_country_investements(movies_df)
top5_countries = invesments_by_country.head(5)
print(top5_countries)
#top5_countries.to_csv("output_q2.csv")

country
United States of America    120153886644
France                        2256831838
United Kingdom                1611604610
India                         1169682797
Japan                          832585873
Name: budget, dtype: int64


#### Resolucion Q3: Películas de Producción Argentina estrenadas a partir del 2000, con mayor y con menor promedio de rating.

In [21]:
def rating_arg_movies(movies_Arg_2000_onwards_df,ratings_df):
    movies_Arg_2000_onwards_df = movies_Arg_2000_onwards_df.astype({'id': int})
    ranking_arg_post_2000_df = movies_Arg_2000_onwards_df[["id", "title"]].merge(ratings_df,
                                                                                left_on="id",
                                                                                right_on="movieId")
    mean_ranking_arg_post_2000_df = ranking_arg_post_2000_df.groupby(["id", "title"])['rating'].mean().reset_index()

    return mean_ranking_arg_post_2000_df

In [22]:
movies_Arg_production = filter_by_country(["Argentina"], movies_df)
movies_Arg_2000_onwards_df = filter_by_year(2000, 0, movies_Arg_production)
mean_ranking_arg_post_2000_df = rating_arg_movies(movies_Arg_2000_onwards_df, ratings_df)

In [23]:
#Max
mean_ranking_arg_post_2000_df.iloc[mean_ranking_arg_post_2000_df['rating'].idxmax()]

id                         125619
title     The forbidden education
rating                        4.0
Name: 23, dtype: object

In [24]:
#Min
mean_ranking_arg_post_2000_df.iloc[mean_ranking_arg_post_2000_df['rating'].idxmin()]

id               128598
title     Left for Dead
rating              1.0
Name: 27, dtype: object

#### Resolucion Q4: Top 10 de actores con mayor participación en películas de producción Argentina con fecha de estreno posterior al 2000

In [25]:
import ast

def actors_with_most_participation_Arg(movies_Arg_2000_onwards_df, credits_df):
    # merge
    actors_in_Arg_movies_2000_df = movies_Arg_2000_onwards_df[["id", "title"]].merge(credits_df, on="id")
    actors_in_Arg_movies_2000_df["cast"] = actors_in_Arg_movies_2000_df["cast"].apply(ast.literal_eval)
    # actors = dict()
    # for cast_in_movie in actors_in_Arg_movies_2000_df["cast"]:
    #     for actor in cast_in_movie:
    #         actors[actor] = actors.get(actor, 0) + 1

    # return actors

    cast_and_movie_arg_post_2000_df = actors_in_Arg_movies_2000_df.set_index("id")["cast"].apply(pd.Series).stack().reset_index("id", name="name")
    cast_per_movie_quantities = cast_and_movie_arg_post_2000_df.groupby(["name"]).count().reset_index().rename(columns={"id":"count"})
    return cast_per_movie_quantities



In [26]:
# movies_per_actor = actors_with_most_participation_Arg(movies_Arg_2000_onwards_df, credits_df)
# sorted_actors = sorted(movies_per_actor.items(), key=lambda x: x[1], reverse=True)

In [27]:
cast_per_movie = actors_with_most_participation_Arg(movies_Arg_2000_onwards_df, credits_df)
cast_per_movie.nlargest(10, 'count')

,name,count
1010,Ricardo Darín,17
47,Alejandro Awada,7
489,Inés Efron,7
664,Leonardo Sbaraglia,7
1125,Valeria Bertuccelli,7
111,Arturo Goetz,6
285,Diego Peretti,6
948,Pablo Echarri,6
998,Rafael Spregelburd,6
1033,Rodrigo de la Serna,6


In [ ]:
#cast_per_movie.nlargest(10, 'count').to_csv("output_q4.csv")

#### Resolucion Q5: Average de la tasa ingreso/presupuesto de peliculas con overview de sentimiento positivo vs. sentimiento negativo.

In [ ]:
q5_input_df = movies_df.copy()
q5_input_df.shape


(44435, 9)

In [30]:
# Nos quedamos con rows válidas
q5_input_df = q5_input_df.loc[q5_input_df['budget'] != 0]
q5_input_df = q5_input_df.loc[q5_input_df['revenue'] != 0]
q5_input_df.shape

(5370, 9)

In [32]:
# Cargar modelo preentrenado para análisis de sentimiento
sentiment_analyzer = pipeline('sentiment-analysis', model='distilbert-base-uncased-finetuned-sst-2-english')

Device set to use cpu


In [34]:
start_time = time.time()

# Truncate the text to 512 tokens (or approximately 512 characters as a safe limit)
q5_input_df['sentiment'] = q5_input_df['overview'].fillna('').apply(
	lambda x: sentiment_analyzer(x[:512])[0]['label']
)

elapsed_time = time.time() - start_time
print(f"Execution time: {elapsed_time:.2f} seconds")

Execution time: 366.01 seconds


In [44]:
q5_input_df["rate_revenue_budget"] = q5_input_df["revenue"] / q5_input_df["budget"]

In [37]:
q5_input_df.sample(3)

,id,title,genres,release_date,overview,production_countries,spoken_languages,budget,revenue,sentiment,rate_revenue_budget
35592,351072,Na Maloom Afraad,"['Thriller', 'Comedy']",2014-10-06,"Na Maloom Afraad is a story of three reckless poor struggling souls, running after their individ...",[],['اردو'],884130,2259444.0,POSITIVE,2.555556
33308,257445,Goosebumps,"['Adventure', 'Horror', 'Comedy']",2015-08-05,A teenager teams up with the daughter of young adult horror author R.L. Stine after the writer's...,"['Australia', 'United States of America']",['English'],58000000,158162788.0,POSITIVE,2.726945
11889,675,Harry Potter and the Order of the Phoenix,"['Adventure', 'Fantasy', 'Family', 'Mystery']",2007-06-28,"Returning for his fifth year of study at Hogwarts, Harry is stunned to find that his warnings ab...","['United Kingdom', 'United States of America']",['English'],150000000,938212738.0,POSITIVE,6.254752


In [45]:
average_rate_by_sentiment = q5_input_df.groupby("sentiment")["rate_revenue_budget"].mean()
print(average_rate_by_sentiment)

sentiment
NEGATIVE    5408.329049
POSITIVE    5703.695244
Name: rate_revenue_budget, dtype: float64
